In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentimenttt/TrainingSet.csv
/kaggle/input/sentimenttt/TestSet.csv


In [3]:
# Import necessary libraries
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Masking
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, BatchNormalization, 
    Bidirectional, Reshape
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding

In [4]:
data = pd.read_csv('/kaggle/input/sentimenttt/TrainingSet.csv')
data

,Unnamed: 0,Comment,Sentiment
0,0,Este produto com o novo tipo de bateria de lit...,Negative
1,1,très déçue!! L’empreinte et le tactile ne fonc...,Negative
2,2,Fui a pasar un fin de semana y decidí este hot...,Negative
3,3,"Para ser un hotel del 4 estrellas, deja bastan...",Negative
4,4,"Кормят вкусно, еда разнообразная, фрукты на уж...",Neutral
...,...,...,...
28735,29295,Un très beau stylo caméra le son et l'image so...,Positive
28736,29296,"пришло быстро,но маломерка в ширину рост подошел",Positive
28737,29297,Ne vaut pas plus que le prix on verra la durée,Neutral
28738,29298,Нормальный картофель,Neutral


In [5]:
!pip install transformers datasets torch pandas scikit-learn xgboost nltk tqdm


In [6]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  # For lemmatization
nltk.download('omw-1.4')  


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

Preprocessing in 2 steps ( 2 cells below) for better accuracy

In [7]:

!pip install tqdm

# Import tqdm and enable it for pandas
from tqdm import tqdm
tqdm.pandas() 

# Load dataset
data = pd.read_csv("/kaggle/input/sentimenttt/TrainingSet.csv")  
data = data.dropna(subset=['Comment'])  # Remove missing values


def preprocess_text(text):
    if pd.isna(text):
        return ""
    # Preserve original characters, only lowercasing and whitespace cleanup
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single
    return text.strip().lower()

# Apply preprocessing
data['Processed_Comment'] = data['Comment'].apply(preprocess_text)

#  Lemmatization 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    if not text or pd.isna(text):
        return ""
    lemmatized_words = []
    for w in w_tokenizer.tokenize(text):
        try:
            lemma = lemmatizer.lemmatize(w)
            lemmatized_words.append(lemma)
        except:
            lemmatized_words.append(w)
    return " ".join(lemmatized_words)

# Apply lemmatization with progress bar
data['Processed_Comment'] = data['Processed_Comment'].progress_apply(lemmatize_text)


print(data.head())

100%|██████████| 28740/28740 [12:03<00:00, 39.70it/s]

   Unnamed: 0                                            Comment Sentiment  \
0           0  Este produto com o novo tipo de bateria de lit...  Negative   
1           1  très déçue!! L’empreinte et le tactile ne fonc...  Negative   
2           2  Fui a pasar un fin de semana y decidí este hot...  Negative   
3           3  Para ser un hotel del 4 estrellas, deja bastan...  Negative   
4           4  Кормят вкусно, еда разнообразная, фрукты на уж...   Neutral   

                                   Processed_Comment  
0  este produto com o novo tipo de bateria de lit...  
1  très déçue!! l’empreinte et le tactile ne fonc...  
2  fui a pasar un fin de semana y decidí este hot...  
3  para ser un hotel del 4 estrellas, deja bastan...  
4  кормят вкусно, еда разнообразная, фрукты на уж...  


In [8]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

# Lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemma_processing(text):
    if not text or pd.isna(text):
        return ""
    lemmatized_words = []
    for w in w_tokenizer.tokenize(text):
        try:
            lemma = lemmatizer.lemmatize(w)
            lemmatized_words.append(lemma)
        except:
            lemmatized_words.append(w)
    return " ".join(lemmatized_words)

# Apply preprocessing and lemmatization
data['Processed_Comment'] = data['Comment'].apply(clean_text)
data['Processed_Comment'] = data['Processed_Comment'].progress_apply(lemma_processing)

100%|██████████| 28740/28740 [12:03<00:00, 39.73it/s]


Used Roberta model in below cell for only embedding generation(as said in rules) but not for prediction****

In [11]:
import torch
model_name = "xlm-roberta-base"  # Enhanced multilingual model for pretrained embedding only
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

#  Embedding Generation 
def batch_embeddings(texts, batch_size=32):
    bert_model.eval()
    all_embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size].tolist()
        inputs = tokenizer(
            batch, 
            padding=True, 
            truncation=True, 
            max_length=128, 
            return_tensors="pt"
        ).to(device)
        
        with torch.no_grad():
            outputs = bert_model(**inputs)
        
       
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(cls_embeddings)
    
    return np.concatenate(all_embeddings, axis=0)


In [12]:
# Generate embeddings
embeddings = batch_embeddings(data['Processed_Comment'])


label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['Sentiment'])

100%|██████████| 899/899 [03:04<00:00,  4.88it/s]


In [13]:
#  Split dataset
X_train, X_val, y_train, y_val = train_test_split(
    embeddings, 
    labels, 
    test_size=0.2, 
    random_state=42,
    stratify=labels  
)

# Reshape embeddings  model input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])

In [14]:
# Define Neural Network Model
def create_advanced_model(input_shape, num_classes):
    model = Sequential([
        # Input layer
        Dense(1024, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.4),

        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        # Output layer with softmax for multi-class classification
        Dense(num_classes, activation='softmax')
    ])
    
    return model

In [15]:

input_shape = (X_train.shape[1],)  # Input shape for dense layers
num_classes = len(label_encoder.classes_)

# Create and Compile Model using Optimizer
model = create_advanced_model(input_shape, num_classes)
model.compile(
    optimizer=Adam(learning_rate=0.0003, clipvalue=1.0), 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,  # Increased patience
    restore_best_weights=True,
    min_delta=0.0005
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.4,  
    patience=7,
    min_lr=0.000005
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Model Training 
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=150,  
    batch_size=64,  
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

Epoch 1/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5050 - loss: 1.2197 - val_accuracy: 0.6129 - val_loss: 0.8186 - learning_rate: 3.0000e-04
Epoch 2/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6762 - loss: 0.7602 - val_accuracy: 0.7281 - val_loss: 0.6337 - learning_rate: 3.0000e-04
Epoch 3/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6943 - loss: 0.6986 - val_accuracy: 0.7354 - val_loss: 0.5984 - learning_rate: 3.0000e-04
Epoch 4/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7089 - loss: 0.6657 - val_accuracy: 0.7512 - val_loss: 0.5847 - learning_rate: 3.0000e-04
Epoch 5/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7191 - loss: 0.6492 - val_accuracy: 0.7498 - val_loss: 0.5844 - learning_rate: 3.0000e-04
Epoch 6/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7233 - loss: 0.6235 - val_accuracy: 0.7444 - val_loss: 0.5886 - learning_rate: 3.0000e-04
Epoch 7/150
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - a

In [17]:
# Evaluation
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.7634


Processing of test data described below

In [18]:
# Load Test Dataset
test_data = pd.read_csv("/kaggle/input/sentimenttt/TestSet.csv")
test_data = test_data.dropna(subset=['Comment'])

# Preprocess test data
test_data['Processed_Comment'] = test_data['Comment'].apply(preprocess_text)

# Generate embeddings for test data
test_embeddings = batch_embeddings(test_data['Processed_Comment'])

# Reshape test embeddings for model input
test_embeddings = test_embeddings.reshape(test_embeddings.shape[0], test_embeddings.shape[1])

# Make predictions
test_predictions = model.predict(test_embeddings)
test_pred_labels = np.argmax(test_predictions, axis=1)

# Convert predictions back to original labels
predicted_labels = label_encoder.inverse_transform(test_pred_labels)

# Create submission DataFrame
submission = pd.DataFrame({
    'index': test_data.index,
    'sentiment': predicted_labels
})

# Save submission to CSV
output_path = "/kaggle/working/submission.csv"
submission.to_csv(output_path, index=False)

print(f"Submission file saved to {output_path}")
print(submission.head())


print("\nPredicted Sentiment Distribution:")
print(submission['sentiment'].value_counts(normalize=True))

100%|██████████| 452/452 [01:32<00:00,  4.91it/s]


452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Submission file saved to /kaggle/working/submission.csv
   index sentiment
0      0  Negative
1      1   Neutral
2      2  Positive
3      3  Negative
4      4   Neutral

Predicted Sentiment Distribution:
sentiment
Positive    0.503457
Negative    0.292906
Neutral     0.203637
Name: proportion, dtype: float64
